In [1]:
import numpy as np
import torch
import json

from torch_geometric.data import Batch
from libraries.model      import nGCNN, eGCNN, denoise, get_random_graph
from libraries.dataset    import revert_standardize_dataset
from libraries.graph      import POSCAR_graph_encoding

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# From random noise, we generate completely new materials
# A target property can be seeked with this approach

In [3]:
# Define folder in which all data will be stored
target_folder    = 'models/QM9-dsC7O2H10nsd/GM_v0'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Number of graphs to predict
N_predictions = 10

# Define target to be generated
target_tensor = None

# Load model data

In [4]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [5]:
labels_name                 = f'{target_folder}/labels.pt'
dataset_name                = f'{target_folder}/dataset.pt'
dataset_name_std            = f'{target_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset = torch.load(dataset_name_std)
labels  = torch.load(labels_name)

# Read the file in JSON format to a dictionary
with open(dataset_parameters_name_std, 'r') as json_file:
    dataset_parameters = json.load(json_file)

In [6]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

# Loading the model

In [7]:
for graph in dataset:
    #graph.y = torch.tensor([graph.y], dtype=torch.float)
    graph.y = torch.tensor([0], dtype=torch.float)

In [8]:
# Determine number of node-level features in dataset, considering the t_step information
n_node_features = dataset[0].num_node_features + 1

# Determine the number of graph-level features to be predicted
n_graph_features = len(dataset[0].y)

# Instantiate the models for nodes and edges
node_model = nGCNN(n_node_features, n_graph_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = eGCNN(n_node_features, n_graph_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)

# Generating new cystals

In [9]:
# Predicting loop
diffused_batch = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Get random graph, acting as diffused
        diffused_graph = get_random_graph(n_nodes, n_node_features-1)
        diffused_graph.y = torch.tensor([0], dtype=torch.float)
        diffused_batch.append(diffused_graph)

    # Generate batch objects
    diffused_batch = Batch.from_data_list(diffused_batch)
    
    # Move data to device
    diffused_batch = diffused_batch.to(device)
    
    # Denoise batch of diffused graphs
    #print(f'Denoising...')
    predicted_dataset = denoise(diffused_batch, n_t_steps, node_model, edge_model, n_graph_features,
                                s=alpha_decay, sigma=model_parameters['sigma'])

# From batch object to list
predicted_dataset = predicted_dataset.to_data_list()

# Revert stardadization
denoised_graphs = revert_standardize_dataset(predicted_dataset, dataset_parameters)
denoised_graphs

In [11]:
temp = denoised_graphs[0]

nodes = temp.x
edges = temp.edge_index.detach().cpu().numpy().T
attributes = temp.edge_attr.detach().cpu().numpy()

In [12]:
import numpy as np
import torch

from scipy.optimize       import minimize
from libraries.graph      import graph_POSCAR_encoding, find_closest_key, composition_concentration_from_keys
from torch_geometric.data import Data
from pymatgen.core        import Structure


is_molecule = True


if is_molecule:
    # Initial guess for the positions
    solution = np.random.rand(len(nodes) * 3)*10  # Initialize all points at origin, 1D array
    #solution = coordinates.reshape(-1, 1).ravel()
    
    # Function to calculate the squared difference between distances and weights
    def objective(solution_attempt, edges, weights):
        positions = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
        errors = 0
        for edge, weight in zip(edges, weights):
            p1 = positions[edge[0]]
            p2 = positions[edge[1]]
            distance = np.linalg.norm(p2 - p1)
            errors += np.power(distance - weight, 2)
        print(errors)
        return errors
    
    def worst_identification(edges, attributes, solution_attempt):
        positions = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
    
        particle_errors = []
        for particle in np.unique(edges):
            # Get those edge indexes where particle has a connection
            particle_connections = np.where((edges[:, 0] == particle) | (edges[:, 1] == particle))
    
            particle_error = 0
            for idx in particle_connections[0]:
                # Load indexes in edge
                edge = edges[idx]
    
                # Load expected attribute
                p1 = positions[edge[0]]
                p2 = positions[edge[1]]
    
                # Load reference attribute
                weight = attributes[idx].item()
                
                # Compute error
                distance = np.linalg.norm(p2 - p1)

                # Append to trial errors for different atom images
                trial_error = np.power(distance - weight, 2)
    
                # Add error
                particle_error += trial_error
    
            # Average over the connection of the node
            particle_error /= len(particle_connections[0])
    
            # Append particle error
            particle_errors.append(particle_error)
    
        return np.argmax(particle_errors), np.max(particle_errors)

In [18]:
error_threshold = 1e-5

n_attempts = 1
for attempt in range(n_attempts):
    print()
    print(f'Attempt {attempt}')
    solution = minimize(objective, solution,
                        args=(edges, attributes),
                        method='Powell')

    is_success       = solution.success
    solution_message = solution.message
    worst_particle, worst_error = worst_identification(edges, attributes, solution.x)

    attempt_error = objective(solution.x, edges, attributes)
    print(f'Total: {attempt_error} and local {worst_error} errors')

    if (attempt_error < error_threshold) or (attempt == n_attempts-1):
        break
    
    solution = solution.x.reshape(-1, 3)  # Reshape to 2D array

    # Re-initialize that position
    solution[worst_particle+3] = np.random.rand(3)

    solution = solution.flatten()

# Check convergence status
if is_success:
    print('Converged to a solution.')
else:
    print(f'Failed to converge: {solution_message}')


solution = solution.x.reshape(-1, 3)  # Reshape to 2D array

In [20]:
if is_molecule:
    # Get the position of each atom in direct coordinates
    #direct_positions = graph_to_cartesian_positions(graph)
    #cartesian_positions = solution.x.reshape(-1, 3)*mw
    #cartesian_positions = solution.x.reshape(-1, 3)
    
    lattice_vectors     = np.array([[100, 0,   0],
                                    [0,   100, 0],
                                    [0,   0,   100]])
    cartesian_positions = solution


POSCAR_name = None

# Get name for the first line of the POSCAR
POSCAR_name = POSCAR_name or 'POSCAR from GenerativeModels'

# Clone the input graph to preserve the original structure
new_graph = temp.clone()

# Load and detach embeddings for the graph nodes
data_embeddings = new_graph.x.detach().cpu().numpy()

# Loading dictionary of available embeddings for atoms
available_embeddings = {}
with open('../MP/input/atomic_masses.dat', 'r') as atomic_masses_file:
    for line in atomic_masses_file:
        key, mass, charge, electronegativity, ionization_energy = line.split()

        # Check if all information is present
        if all(val != 'None' for val in (mass, charge, electronegativity, ionization_energy)):
            available_embeddings[key] = np.array([mass, charge, electronegativity, ionization_energy], dtype=float)

# Get most similar atoms for each graph node and create a list of keys
keys = [find_closest_key(available_embeddings, emb) for emb in data_embeddings]

# Get elements' composition, concentration, and positions
POSCAR_composition, POSCAR_concentration, POSCAR_positions = composition_concentration_from_keys(keys, cartesian_positions)


# Write file
with open('CONTCAR', 'w') as POSCAR_file:
    # Delete previous data in the file
    POSCAR_file.truncate()
    
    # Write POSCAR's name
    POSCAR_file.write(f'{POSCAR_name}\n')

    # Write scaling factor (assumed to be 1.0)
    POSCAR_file.write('1.0\n')

    # Write lattice parameters (assumed to be orthogonal)
    np.savetxt(POSCAR_file, lattice_vectors, delimiter=' ')

    # Write composition (each different species, previously sorted)
    np.savetxt(POSCAR_file, [POSCAR_composition], fmt='%s', delimiter=' ')

    # Write concentration (number of each of the previous elements)
    np.savetxt(POSCAR_file, [POSCAR_concentration], fmt='%d', delimiter=' ')

    # Write position in cartesian form
    POSCAR_file.write('Cartesian\n')
    np.savetxt(POSCAR_file, POSCAR_positions, delimiter=' ')